In [1]:
import pandas as pd
import catboost as cb

In [2]:
# Чтение feed_data из csv файла
feed_data = pd.read_csv(r'C:\Users\Alex\Desktop\data_for_training.csv')
print(f"Feed data shape: {feed_data.shape}")

Feed data shape: (15378483, 51)


In [3]:
# Calculate the memory usage of each column
memory_usage_per_column = feed_data.memory_usage(deep=True)

# Calculate the total memory usage
total_memory_usage_bytes = memory_usage_per_column.sum()

# Convert total memory usage to gigabytes
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)

# Print the memory usage
print(f"Memory usage per column (bytes):\n{memory_usage_per_column}\n")
print(f"Total memory usage (bytes): {total_memory_usage_bytes}")
print(f"Total memory usage (gigabytes): {total_memory_usage_gb:.10f}")

Memory usage per column (bytes):
Index                           128
user_id                   123027864
post_id_x                 123027864
action                    938087463
target                    123027864
gender                    123027864
age                       123027864
country                   123027864
city                      123027864
exp_group                 123027864
os                        123027864
source                    123027864
topic                     123027864
word_count                123027864
sentence_count            123027864
avg_word_length           123027864
punctuation_count         123027864
month                     123027864
day_of_week               123027864
hour_of_day               123027864
time_since_last_action    123027864
day_of_month              123027864
year                      123027864
user_views                123027864
user_likes                123027864
post_views                123027864
post_likes                12302

In [4]:
feed_data.columns

Index(['user_id', 'post_id_x', 'action', 'target', 'gender', 'age', 'country',
       'city', 'exp_group', 'os', 'source', 'topic', 'word_count',
       'sentence_count', 'avg_word_length', 'punctuation_count', 'month',
       'day_of_week', 'hour_of_day', 'time_since_last_action', 'day_of_month',
       'year', 'user_views', 'user_likes', 'post_views', 'post_likes',
       '0_exp_group_likes', '0_exp_group_views', '1_exp_group_likes',
       '1_exp_group_views', '2_exp_group_likes', '2_exp_group_views',
       '3_exp_group_likes', '3_exp_group_views', '4_exp_group_likes',
       '4_exp_group_views', '5_exp_group_likes', '5_exp_group_views',
       '6_exp_group_likes', '6_exp_group_views', 'component_1', 'component_2',
       'component_3', 'component_4', 'component_5', 'component_6',
       'component_7', 'component_8', 'component_9', 'component_10',
       'post_id_y'],
      dtype='object')

In [5]:
def sample_10_percent(group):
    frac = 0.1
    return group.sample(frac=frac)

feed_data = feed_data.groupby('user_id', group_keys=False).apply(sample_10_percent)

In [6]:
# Calculate the memory usage of each column
memory_usage_per_column = feed_data.memory_usage(deep=True)

# Calculate the total memory usage
total_memory_usage_bytes = memory_usage_per_column.sum()

# Convert total memory usage to gigabytes
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)

# Print the memory usage
print(f"Memory usage per column (bytes):\n{memory_usage_per_column}\n")
print(f"Total memory usage (bytes): {total_memory_usage_bytes}")
print(f"Total memory usage (gigabytes): {total_memory_usage_gb:.10f}")

Memory usage per column (bytes):
Index                     12303024
user_id                   12303024
post_id_x                 12303024
action                    93810558
target                    12303024
gender                    12303024
age                       12303024
country                   12303024
city                      12303024
exp_group                 12303024
os                        12303024
source                    12303024
topic                     12303024
word_count                12303024
sentence_count            12303024
avg_word_length           12303024
punctuation_count         12303024
month                     12303024
day_of_week               12303024
hour_of_day               12303024
time_since_last_action    12303024
day_of_month              12303024
year                      12303024
user_views                12303024
user_likes                12303024
post_views                12303024
post_likes                12303024
0_exp_group_likes     

In [9]:
feed_data.shape

(1537878, 51)

In [10]:
X = feed_data.drop(['action', 'target'], axis=1)

In [11]:
# Calculate the memory usage of each column
memory_usage_per_column = X.memory_usage(deep=True)

# Calculate the total memory usage
total_memory_usage_bytes = memory_usage_per_column.sum()

# Convert total memory usage to gigabytes
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)

# Print the memory usage
print(f"Memory usage per column (bytes):\n{memory_usage_per_column}\n")
print(f"Total memory usage (bytes): {total_memory_usage_bytes}")
print(f"Total memory usage (gigabytes): {total_memory_usage_gb:.10f}")

Memory usage per column (bytes):
Index                     12303024
user_id                   12303024
post_id_x                 12303024
gender                    12303024
age                       12303024
country                   12303024
city                      12303024
exp_group                 12303024
os                        12303024
source                    12303024
topic                     12303024
word_count                12303024
sentence_count            12303024
avg_word_length           12303024
punctuation_count         12303024
month                     12303024
day_of_week               12303024
hour_of_day               12303024
time_since_last_action    12303024
day_of_month              12303024
year                      12303024
user_views                12303024
user_likes                12303024
post_views                12303024
post_likes                12303024
0_exp_group_likes         12303024
0_exp_group_views         12303024
1_exp_group_likes     

Обрезали на 2.8 гига, теперь заливаем в SQL

In [12]:
from tqdm import tqdm
import math
import pandas as pd
from sqlalchemy import create_engine
import time

def upload_dataframe_in_chunks(data, table_name, engine, chunksize=10000):
    total_chunks = math.ceil(len(data) / chunksize)
    for i in tqdm(range(total_chunks), desc=f"Uploading to {table_name}"):
        chunk = data[i * chunksize : (i + 1) * chunksize]
        if_exists = "replace" if i == 0 else "append"
        chunk.to_sql(table_name, con=engine, if_exists=if_exists, index=False, method="multi")


engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)
chunksize = 10000
upload_dataframe_in_chunks(X, "a-efimik_features_lesson_22_500MB", engine, chunksize=chunksize)

Uploading to a-efimik_features_lesson_22_500MB: 100%|██████████| 154/154 [1:05:06<00:00, 25.36s/it]


In [13]:
X.head()

,user_id,post_id_x,gender,age,country,city,exp_group,os,source,topic,...,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,post_id_y
38,200,1936,1,34,7,651,3,0,0,6,...,-0.079554,-0.041727,-0.012298,-0.041089,0.092944,-0.013891,0.027997,0.007086,0.017944,1936
5,200,1569,1,34,7,651,3,0,0,5,...,-0.025003,0.014029,0.004516,-0.104723,-0.028520,-0.094245,-0.031559,0.105480,0.085020,1569
39,200,6697,1,34,7,651,3,0,0,3,...,0.040700,-0.017044,0.000160,-0.042450,0.028215,0.103546,-0.038677,0.018742,0.018075,6697
67,200,6306,1,34,7,651,3,0,0,3,...,0.113883,-0.014311,0.009417,0.014441,-0.008787,-0.056462,-0.025536,0.003592,-0.061648,6306
63,200,6919,1,34,7,651,3,0,0,3,...,0.047569,-0.022153,-0.018576,-0.055399,0.023171,0.106228,-0.038756,-0.021442,0.008517,6919
